In [58]:
# BFS - Breadth First Search

from collections import deque

graph = {
    'S': ['A', 'B'],
    'A': ['C'],
    'B': ['C', 'G'],
    'C': ['D'],
    'D': ['G'],
    'G': []
}

def BFS(graph,start,goal):
    # Visited
    visited=[start]
    # Double Ended Queue
    dq=deque()
    # Path to track shortest path
    path=[start]
    # Appending root node of the graph and status=>visited
    dq.append((start,path))

    #Iterating till deque is empty
    while dq:
        # Popping out first node from the queue
        (node,path) = dq.popleft()

        if node==goal:
            return path

        # Iterating through the neighbors of the node
        for neighbor in graph[node]:
           if neighbor not in visited:
                visited.append(neighbor) # Status=>visited
                new_path = path + [neighbor] # Create new shortest path from the root
                dq.append((neighbor,new_path)) # Add the node to the deque
    return None

def BFS_me(graph,start,goal):
    visited=[start]
    dq = deque()
    dq.append((start,[start]))
    
    while dq:
        (node,path) = dq.popleft()
        if node==goal:
            return path
        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.append(neighbor)
                new_path = path + [neighbor]
                dq.append((neighbor,new_path))
    return None
    
print("Path to the goal node :",BFS_me(graph,'S','G'))

Path to the goal node : ['S', 'B', 'G']


In [59]:
# DFS - Depth First Search

def DFS(graph,start,goal,path=[]):
    visited=[]
    sk = []
    sk.append((start,[start]))
    visited.append(start)

    while sk:
        (node,path) = sk.pop()
        print(node,path)
        if node==goal:
            return path
        for neighbor in reversed(graph[node]):
            if neighbor not in visited:
                visited.append(neighbor)
                new_path = path + [neighbor]
                sk.append((neighbor,new_path))
                
    return None

def DFS_me(graph,start,goal):
    visited=[]
    sk = []
    sk.append((start,[start]))
    visited.append(start)

    while sk:
        (node,path) = sk.pop()
        if goal==node:
            return path
        for neighbor in reversed(graph[node]):
            if neighbor not in visited:
                visited.append(neighbor)
                new_path = path + [neighbor]
                sk.append((neighbor,new_path))
    return None
            
print(DFS_me(graph,'S','G'))

['S', 'A', 'C', 'D', 'G']


In [68]:
# DLS - Depth Limited Search

def DLS_me(graph,start,goal,depth):
    visited=[]
    sk = []
    sk.append((start,[start]))
    visited.append(start)
    now_depth=0
    while(sk and now_depth<depth+1):
        now_depth+=1
        (node,path) = sk.pop()
        if node==goal:
            return path
        for neighbor in reversed(graph[node]):
            visited.append(neighbor)
            new_path = path + [neighbor]
            sk.append((neighbor,new_path))
    return None

DLS_me(graph,'S','G',4)

['S', 'A', 'C', 'D', 'G']

In [72]:
# IDDFS - Iterative Deepening Depth First Search
def IDDFS(graph,start,goal,maxdepth):
    for i in range(maxdepth):
        path = DLS_me(graph,start,goal,i)
        if path:
            return path,i

IDDFS(graph,'S','G',10)

(['S', 'A', 'C', 'D', 'G'], 4)

In [94]:
# Uniform Cost Search
graph = {
    'A': [('B', 1), ('C', 4)],
    'B': [('E', 1)],
    'C': [('F', 5)],
    'E': [('G', 1)],
    'F': [('E', 2)],  # Note the higher cost here
    'G': []          # Goal node has no outgoing edges
}

import heapq

def UCS(graph,start,goal):
    visited=[]
    pq = []
    heapq.heappush(pq,(0,(start,[start])))
    visited.append(start)
    while pq:
        (pri,N) = heapq.heappop(pq)
        (node,path) = N
        print(node)
        if node == goal:
            return path
        for neighbor,priority in graph[node]:
            print(neighbor,priority)
            if neighbor not in visited:
                visited.append(node)
                new_path = path + [neighbor]
                new_pri = priority + pri
                heapq.heappush(pq,(new_pri,(neighbor,new_path)))
    return None
UCS(graph,'A','G')

A
B 1
C 4
B
E 1
E
G 1
G


['A', 'B', 'E', 'G']

In [3]:
# Bi-Directional Search
# Unweighted, undirected graph
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'E'],
    'C': ['A', 'F'],
    'E': ['B', 'F', 'G'],
    'F': ['C', 'E'],
    'G': ['E']
}


from collections import deque

def BDS_with_BFS(graph, start, goal):
    # Check for trivial cases
    if start == goal:
        return [start]
    
    # Forward search data structures
    queue_fwd = deque([(start, [start])])
    visited_fwd = {start}
    
    # Backward search data structures
    queue_bwd = deque([(goal, [goal])])
    visited_bwd = {goal}
    
    # Parents for path reconstruction
    parent_fwd = {start: None}
    parent_bwd = {goal: None}
    
    while queue_fwd and queue_bwd:
        # ---- Forward search step ----
        current_fwd, path_fwd = queue_fwd.popleft()
        
        # Check for intersection after moving forward
        if current_fwd in visited_bwd:
            return reconstruct_path(parent_fwd, parent_bwd, current_fwd)

        for neighbor in graph.get(current_fwd, []):
            if neighbor not in visited_fwd:
                visited_fwd.add(neighbor)
                parent_fwd[neighbor] = current_fwd
                queue_fwd.append((neighbor, path_fwd + [neighbor]))
        
        # ---- Backward search step ----
        current_bwd, path_bwd = queue_bwd.popleft()

        # Check for intersection after moving backward
        if current_bwd in visited_fwd:
            return reconstruct_path(parent_fwd, parent_bwd, current_bwd)

        for neighbor in graph.get(current_bwd, []):
            if neighbor not in visited_bwd:
                visited_bwd.add(neighbor)
                parent_bwd[neighbor] = current_bwd
                queue_bwd.append((neighbor, path_bwd + [neighbor]))
                
    return None # No path found

def reconstruct_path(parent_fwd, parent_bwd, meeting_node):
    # Reconstruct path from start to meeting node
    path_fwd = []
    current = meeting_node
    while current is not None:
        path_fwd.append(current)
        current = parent_fwd.get(current)
    path_fwd.reverse()
    
    # Reconstruct path from goal to meeting node
    path_bwd = []
    current = meeting_node
    while current is not None:
        path_bwd.append(current)
        current = parent_bwd.get(current)
        
    # Combine the two paths, removing the duplicate meeting node
    return path_fwd + path_bwd[1:]

# Execute the BDS with BFS
print(BDS_with_BFS(graph, 'A', 'G'))

['A', 'B', 'E', 'G']
